In [11]:
# Comment importer PySpark et vérifier la version ?
# !pip install findspark
# import findspark
# findspark.init()
# !pip install pyspark

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Exercices").getOrCreate()

In [12]:
spark

In [13]:
print(spark.version)

3.5.0


2. Comment convertir l'index d'un PySpark DataFrame en colonne ?

In [14]:
df = spark.createDataFrame([
("Alice", 1),
("Bob", 2),
("Charlie", 3),
], ["Name", "Value"])

df.show()

+-------+-----+
|   Name|Value|
+-------+-----+
|  Alice|    1|
|    Bob|    2|
|Charlie|    3|
+-------+-----+



In [57]:
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql.window import Window

w = Window.orderBy(monotonically_increasing_id())

df = df.withColumn("index", row_number().over(w) - 1)

df.show()

+-------+-----+-----+
|   Name|Value|index|
+-------+-----+-----+
|  Alice|    1|    0|
|    Bob|    2|    1|
|Charlie|    3|    2|
+-------+-----+-----+



3. Comment combiner plusieurs listes pour former un PySpark DataFrame ?

In [42]:
# Define your lists
list1 = ["a", "b", "c", "d"]
list2 = [1, 2, 3, 4]

rdd = spark.sparkContext.parallelize(list(zip(list1, list2)))

rdd.toDF(["colonne1", "colonne2"]).show()

+--------+--------+
|colonne1|colonne2|
+--------+--------+
|       a|       1|
|       b|       2|
|       c|       3|
|       d|       4|
+--------+--------+



4. Comment récupérer les éléments de la liste A non présents dans la liste B ?

In [43]:
list_A = [1, 2, 3, 4, 5]
list_B = [4, 5, 6, 7, 8]

In [47]:
rdd1 = spark.sparkContext.parallelize(list_A)
rdd2 = spark.sparkContext.parallelize(list_B)

In [48]:
rdd1.subtract(rdd2).collect()

[1, 2, 3]

5. Comment obtenir les éléments qui ne sont pas communs à la liste A et à la liste B ?

In [58]:
list_A = [1, 2, 3, 4, 5]
list_B = [4, 5, 6, 7, 8]

rdd1 = spark.sparkContext.parallelize(list_A)
rdd2 = spark.sparkContext.parallelize(list_B)

rdd_1 = rdd1.subtract(rdd2)
rdd_2 = rdd2.subtract(rdd1)

rdd_union = rdd_1.union(rdd_2)

print(rdd_union.collect())

[1, 2, 3, 8, 6, 7]


6. Comment obtenir le minimum, le 25e centile, la médiane, le 75e et le maximum d'une colonne numérique ?

In [60]:
# Create a sample DataFrame
data = [("A", 10), ("B", 20), ("C", 30), ("D", 40), ("E", 50), ("F", 15), ("G", 28), ("H", 54), ("I", 41), ("J", 86)]
df = spark.createDataFrame(data, ["Name", "Age"])

df.show()

+----+---+
|Name|Age|
+----+---+
|   A| 10|
|   B| 20|
|   C| 30|
|   D| 40|
|   E| 50|
|   F| 15|
|   G| 28|
|   H| 54|
|   I| 41|
|   J| 86|
+----+---+



In [69]:
from pyspark.sql.functions import mean, median, max, min, percentile

df.agg(min("Age").alias("min_age"),\
       percentile("Age", 0.25).alias("median_25"),\
       percentile("Age", 0.5).alias("median_50"),\
       percentile("Age", 0.75).alias("median_75"),\
       max("Age").alias("max_age")
       ).show()

+-------+---------+---------+---------+-------+
|min_age|median_25|median_50|median_75|max_age|
+-------+---------+---------+---------+-------+
|     10|     22.0|     35.0|    47.75|     86|
+-------+---------+---------+---------+-------+



In [77]:
quantiles = df.approxQuantile("Age", [0.0, 0.25, 0.5, 0.75, 1.0], 0.01)

print("min :",quantiles[0])
print("25th :", quantiles[1])
print("50th :", quantiles[0])
print("75th :", quantiles[3])
print("max :", quantiles[4])

min : 10.0
25th : 20.0
50th : 10.0
75th : 50.0
max : 86.0


7. Comment obtenir la fréquence des éléments uniques d’une colonne ?

In [78]:
from pyspark.sql import Row

# Sample data
data = [
Row(name='John', job='Engineer'),
Row(name='John', job='Engineer'),
Row(name='Mary', job='Scientist'),
Row(name='Bob', job='Engineer'),
Row(name='Bob', job='Engineer'),
Row(name='Bob', job='Scientist'),
Row(name='Sam', job='Doctor'),
]

# create DataFrame
df = spark.createDataFrame(data)

# show DataFrame
df.show()

+----+---------+
|name|      job|
+----+---------+
|John| Engineer|
|John| Engineer|
|Mary|Scientist|
| Bob| Engineer|
| Bob| Engineer|
| Bob|Scientist|
| Sam|   Doctor|
+----+---------+



In [81]:
print(df.groupBy("name").count().show())
print(df.groupBy("job").count().show())

+----+-----+
|name|count|
+----+-----+
|Mary|    1|
|John|    2|
| Bob|    3|
| Sam|    1|
+----+-----+

None
+---------+-----+
|      job|count|
+---------+-----+
|Scientist|    2|
| Engineer|    4|
|   Doctor|    1|
+---------+-----+

None


8. Comment conserver telles quelles les deux valeurs les plus fréquentes et remplacer tout le reste par « Autre » ?

In [82]:
from pyspark.sql import Row

# Sample data
data = [
Row(name='John', job='Engineer'),
Row(name='John', job='Engineer'),
Row(name='Mary', job='Scientist'),
Row(name='Bob', job='Engineer'),
Row(name='Bob', job='Engineer'),
Row(name='Bob', job='Scientist'),
Row(name='Sam', job='Doctor'),
]

# create DataFrame
df = spark.createDataFrame(data)

# show DataFrame
df.show()

+----+---------+
|name|      job|
+----+---------+
|John| Engineer|
|John| Engineer|
|Mary|Scientist|
| Bob| Engineer|
| Bob| Engineer|
| Bob|Scientist|
| Sam|   Doctor|
+----+---------+



In [85]:
print(df.groupBy("job").count().show())

+---------+-----+
|      job|count|
+---------+-----+
|Scientist|    2|
| Engineer|    4|
|   Doctor|    1|
+---------+-----+

None


In [87]:
df.replace("Doctor", "Autre").show()

+----+---------+
|name|      job|
+----+---------+
|John| Engineer|
|John| Engineer|
|Mary|Scientist|
| Bob| Engineer|
| Bob| Engineer|
| Bob|Scientist|
| Sam|    Autre|
+----+---------+



In [96]:
from pyspark.sql.functions import col, when

top2 = df.groupBy("job").count().orderBy('count', ascending = False).limit(2)\
.select("job").rdd.flatMap(lambda x: x).collect()

df = df.withColumn("job", when(col().'job'isin(top2), col('job')).otherwise('autre'))

df.show()

+----+---------+
|name|      job|
+----+---------+
|John| Engineer|
|John| Engineer|
|Mary|Scientist|
| Bob| Engineer|
| Bob| Engineer|
| Bob|Scientist|
| Sam|    autre|
+----+---------+



9. Comment supprimer des lignes avec des valeurs NA spécifiques à une colonne particulière ?

In [97]:
# Assuming df is your DataFrame
df = spark.createDataFrame([
("A", 1, None),
("B", None, "123" ),
("B", 3, "456"),
("D", None, None),
], ["Name", "Value", "id"])

df.show()

+----+-----+----+
|Name|Value|  id|
+----+-----+----+
|   A|    1|NULL|
|   B| NULL| 123|
|   B|    3| 456|
|   D| NULL|NULL|
+----+-----+----+



In [98]:
df.select(col("Value")).distinct().show()

+-----+
|Value|
+-----+
|    1|
|    3|
| NULL|
+-----+



In [103]:
df.dropna(subset=["Value"]).show()

+----+-----+----+
|Name|Value|  id|
+----+-----+----+
|   A|    1|NULL|
|   B|    3| 456|
+----+-----+----+



10. Comment renommer les colonnes d'un PySpark DataFrame à l'aide de deux listes – l'une contenant les anciens noms de colonnes et l'autre contenant les nouveaux noms de colonnes ?

In [104]:
# suppose you have the following DataFrame
df = spark.createDataFrame([(1, 2, 3), (4, 5, 6)], ["col1", "col2", "col3"])

# old column names
old_names = ["col1", "col2", "col3"]

# new column names
new_names = ["new_col1", "new_col2", "new_col3"]

df.show()


+----+----+----+
|col1|col2|col3|
+----+----+----+
|   1|   2|   3|
|   4|   5|   6|
+----+----+----+



In [107]:
df.toDF(*new_names).show()

+--------+--------+--------+
|new_col1|new_col2|new_col3|
+--------+--------+--------+
|       1|       2|       3|
|       4|       5|       6|
+--------+--------+--------+



In [110]:
for old_name, new_name in zip(old_names, new_names):
  df = df.withColumnRenamed(old_name, new_name)

TypeError: 'list' object cannot be interpreted as an integer

In [109]:
df.show()

+--------+--------+--------+
|new_col1|new_col2|new_col3|
+--------+--------+--------+
|       1|       2|       3|
|       4|       5|       6|
+--------+--------+--------+



11. Comment regrouper une liste numérique en 10 groupes de taille égale ?

In [111]:
from pyspark.sql.functions import rand
from pyspark.ml.feature import Bucketizer

# Create a DataFrame with a single column "values" filled with random numbers
num_items = 100
df = spark.range(num_items).select(rand(seed=42).alias("values"))

df.show(5)

+-------------------+
|             values|
+-------------------+
|  0.619189370225301|
| 0.5096018842446481|
| 0.8325259388871524|
|0.26322809041172357|
| 0.6702867696264135|
+-------------------+
only showing top 5 rows



In [112]:
df.count()

100

In [113]:
# Define the bucket boundaries
num_buckets = 10
quantiles = df.stat.approxQuantile("values", [i/num_buckets for i in range(num_buckets+1)], 0.01)

# Create the Bucketizer
bucketizer = Bucketizer(splits=quantiles, inputCol="values", outputCol="buckets")

# Apply the Bucketizer
df_buck = bucketizer.transform(df)

#Frequency table
df_buck.groupBy("buckets").count().show()

# Show the original and bucketed values
df_buck.show(5)

+-------+-----+
|buckets|count|
+-------+-----+
|    8.0|   10|
|    0.0|    8|
|    7.0|   10|
|    1.0|   10|
|    4.0|   10|
|    3.0|   10|
|    2.0|   10|
|    6.0|   10|
|    5.0|   10|
|    9.0|   12|
+-------+-----+

+-------------------+-------+
|             values|buckets|
+-------------------+-------+
|  0.619189370225301|    4.0|
| 0.5096018842446481|    4.0|
| 0.8325259388871524|    8.0|
|0.26322809041172357|    2.0|
| 0.6702867696264135|    5.0|
+-------------------+-------+
only showing top 5 rows



12. Comment créer un tableau de contingence ?

In [115]:
# Example DataFrame
data = [("A", "X"), ("A", "Y"), ("A", "X"), ("B", "Y"), ("B", "X"), ("C", "X"), ("C", "X"), ("C", "Y")]
df = spark.createDataFrame(data, ["category1", "category2"])

df.show()

+---------+---------+
|category1|category2|
+---------+---------+
|        A|        X|
|        A|        Y|
|        A|        X|
|        B|        Y|
|        B|        X|
|        C|        X|
|        C|        X|
|        C|        Y|
+---------+---------+



In [116]:
df.crosstab("category1", "category2").show()

+-------------------+---+---+
|category1_category2|  X|  Y|
+-------------------+---+---+
|                  B|  1|  1|
|                  C|  2|  1|
|                  A|  2|  1|
+-------------------+---+---+



13. Comment trouver les nombres multiples de 3 dans une colonne ?

In [117]:
from pyspark.sql.functions import rand

# Generate a DataFrame with a single column "id" with 10 rows
df = spark.range(10)

# Generate a random float between 0 and 1, scale and shift it to get a random integer between 1 and 10
df = df.withColumn("random", ((rand(seed=42) * 10) + 1).cast("int"))

# Show the DataFrame
df.show()

+---+------+
| id|random|
+---+------+
|  0|     7|
|  1|     6|
|  2|     9|
|  3|     3|
|  4|     7|
|  5|     9|
|  6|     7|
|  7|     3|
|  8|     3|
|  9|     7|
+---+------+



In [138]:
df_mult = df.withColumn("dividby3", when(col("random") % 3 == 0, 1).otherwise(0))

df_mult.show()

+---+------+--------+
| id|random|dividby3|
+---+------+--------+
|  0|     7|       0|
|  1|     6|       1|
|  2|     9|       1|
|  3|     3|       1|
|  4|     7|       0|
|  5|     9|       1|
|  6|     7|       0|
|  7|     3|       1|
|  8|     3|       1|
|  9|     7|       0|
+---+------+--------+



14. Comment extraire des éléments à des positions données d'une colonne ?

In [139]:
from pyspark.sql.functions import rand

# Generate a DataFrame with a single column "id" with 10 rows
df = spark.range(10)

# Generate a random float between 0 and 1, scale and shift it to get a random integer between 1 and 10
df = df.withColumn("random", ((rand(seed=42) * 10) + 1).cast("int"))

# Show the DataFrame
df.show()

pos = [0, 4, 8, 5]

+---+------+
| id|random|
+---+------+
|  0|     7|
|  1|     6|
|  2|     9|
|  3|     3|
|  4|     7|
|  5|     9|
|  6|     7|
|  7|     3|
|  8|     3|
|  9|     7|
+---+------+



In [143]:
df.where(df.id.isin(pos)).show()

+---+------+
| id|random|
+---+------+
|  0|     7|
|  4|     7|
|  5|     9|
|  8|     3|
+---+------+



15. Comment empiler deux DataFrames verticalement ?

In [144]:
# Create DataFrame for region A
df_A = spark.createDataFrame([("apple", 3, 5), ("banana", 1, 10), ("orange", 2, 8)], ["Name", "Col_1", "Col_2"])
df_A.show()

# Create DataFrame for region B
df_B = spark.createDataFrame([("apple", 3, 5), ("banana", 1, 15), ("grape", 4, 6)], ["Name", "Col_1", "Col_3"])
df_B.show()

+------+-----+-----+
|  Name|Col_1|Col_2|
+------+-----+-----+
| apple|    3|    5|
|banana|    1|   10|
|orange|    2|    8|
+------+-----+-----+

+------+-----+-----+
|  Name|Col_1|Col_3|
+------+-----+-----+
| apple|    3|    5|
|banana|    1|   15|
| grape|    4|    6|
+------+-----+-----+



In [147]:
combained_data = df_A.unionByName(df_B, allowMissingColumns=True)
combained_data.show()

+------+-----+-----+-----+
|  Name|Col_1|Col_2|Col_3|
+------+-----+-----+-----+
| apple|    3|    5| NULL|
|banana|    1|   10| NULL|
|orange|    2|    8| NULL|
| apple|    3| NULL|    5|
|banana|    1| NULL|   15|
| grape|    4| NULL|    6|
+------+-----+-----+-----+



16. Comment calculer l'erreur quadratique moyenne sur une colonne de vérité et une colonne prédite

In [188]:
# Assume you have a DataFrame df with two columns "actual" and "predicted"
# For the sake of example, we'll create a sample DataFrame
data = [(1, 1), (2, 4), (3, 9), (4, 16), (5, 25)]
df = spark.createDataFrame(data, ["actual", "predicted"])

df.show()

+------+---------+
|actual|predicted|
+------+---------+
|     1|        1|
|     2|        4|
|     3|        9|
|     4|       16|
|     5|       25|
+------+---------+



In [154]:
df.printSchema()

root
 |-- actual: long (nullable = true)
 |-- predicted: long (nullable = true)



In [197]:
from pyspark.sql.functions import sqrt, avg, sum

df_s = df.withColumns({"rmse": (df.actual - df.predicted)**2}).select(col("rmse"))

df_s.show()

df_s.printSchema()

df_s.select(sqrt(sum(df_s.rmse))).show()

#"rmse = df.select(sqrt(avg(df.actual-df.predicted))).alias('rmse')

+-----+
| rmse|
+-----+
|  0.0|
|  4.0|
| 36.0|
|144.0|
|400.0|
+-----+

root
 |-- rmse: double (nullable = true)

+------------------+
|   SQRT(sum(rmse))|
+------------------+
|24.166091947189145|
+------------------+



In [159]:
df.select(avg(df.actual-df.predicted)**2).alias('rmse').show()

+-----------------------------------+
|POWER(avg((actual - predicted)), 2)|
+-----------------------------------+
|                               64.0|
+-----------------------------------+

